### Download Dataset from HF
Let's start by downloading the datasets from HuggingFace.

In [58]:
import os
from pathlib import Path
from dotenv import load_dotenv
from huggingface_hub import hf_hub_download, list_repo_files

load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")

if not HF_TOKEN:
    raise ValueError("HF_TOKEN not found. Please set it.")

REPO_ID = "jd0g/nlistral-7b-dataset"
LOCAL_DATA_DIR = Path("data")

def download_dataset():
    os.makedirs(LOCAL_DATA_DIR, exist_ok=True)
    
    all_repo_files = list_repo_files(repo_id=REPO_ID, repo_type="dataset", token=HF_TOKEN)
    excluded_repo_paths = ["download_dataset.py", "upload_dataset.py", "README.md"] 
    files_to_download = [f for f in all_repo_files if f not in excluded_repo_paths]
    
    if not files_to_download:
        if any(LOCAL_DATA_DIR.iterdir()): # Checks if directory is not empty
             print(f"No new files to download from {REPO_ID}. Local directory {LOCAL_DATA_DIR} not empty.")
             return

    for file_path_in_repo in files_to_download:
        try:
            hf_hub_download(
                repo_id=REPO_ID,
                filename=file_path_in_repo,
                repo_type="dataset",
                token=HF_TOKEN,
                local_dir=LOCAL_DATA_DIR,
                local_dir_use_symlinks=False
            )
        except Exception as e:
            print(f"Error downloading {file_path_in_repo}: {e}") 
            
    print(f"Dataset successfully downloaded for {REPO_ID} into {LOCAL_DATA_DIR}/.")

download_dataset()

Dataset successfully downloaded for jd0g/nlistral-7b-dataset into data/.


In [59]:
import pandas as pd
pd.read_csv('data/original_data/sample.csv').head()

,premise,hypothesis,label
0,a Bombay looks uh i guess the closest thing to...,Is it because Halloween cat caricatures and Bo...,1
1,Sailing north to find and claim islands for th...,Columbus traveled all over the world for the s...,1
2,his conviction was later overturned on appeal.,He was found guilty of Conspiracy to Rob but h...,1
3,"In practice, and depending on the language, th...","In practice, and depending on the language, th...",1
4,"You tread on her foot, or pick up her handkerc...","Don't tread on her foot, or pick up her handke...",1


### Quickstart Model Download
Single-Cell set-up for downloading a specific model ablation (best configuration), base model and tokenizer

In [60]:
import os
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig 
from dotenv import load_dotenv

# --- Configuration ---
HUB_REPO_ID = "jd0g/nlistral-7b-qlora"
ADAPTER_NAME_ON_HUB = "nlistral-7b-qlora-ablation1-best" # This is the subfolder name on the Hub
GPU_ID = 0
# ---

load_dotenv()
HF_TOKEN = os.environ.get("HF_TOKEN")

print(f"Loading adapter '{ADAPTER_NAME_ON_HUB}' from Hub repo '{HUB_REPO_ID}'...")

# Download adapter_config.json from the subfolder on the Hub
try:
    adapter_config = PeftConfig.from_pretrained(HUB_REPO_ID, subfolder=ADAPTER_NAME_ON_HUB, token=HF_TOKEN)
    BASE_MODEL_ID = adapter_config.base_model_name_or_path
    print(f"Base model specified in adapter config: {BASE_MODEL_ID}")
except Exception as e:
    print(f"Could not fetch PeftConfig. Error: {e}")
    print(f"Falling back to default BASE_MODEL_ID: mistralai/Mistral-7B-v0.3") # Fallback
    BASE_MODEL_ID = "mistralai/Mistral-7B-v0.3"


#  Configure 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load the base model with quantization
print(f"Loading base model '{BASE_MODEL_ID}' with 4-bit quantization on GPU {GPU_ID}...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=quantization_config,
    device_map={"": GPU_ID},
    trust_remote_code=True,
    token=HF_TOKEN
)
print("Base model loaded.")

# Load the tokenizer directly from the adapter's location on the Hub
print(f"Loading tokenizer from Hub: '{HUB_REPO_ID}' (subfolder: '{ADAPTER_NAME_ON_HUB}')...")
tokenizer = AutoTokenizer.from_pretrained(
    HUB_REPO_ID,
    subfolder=ADAPTER_NAME_ON_HUB, # Specify the subfolder
    use_fast=True,
    padding_side="left",
    token=HF_TOKEN
)
print("Tokenizer loaded.")

# Resize token embeddings if necessary
if len(tokenizer) != base_model.config.vocab_size:
    print(f"Resizing token embeddings of base model from {base_model.config.vocab_size} to {len(tokenizer)}.")
    base_model.resize_token_embeddings(len(tokenizer))

# Apply the PEFT adapter to the base model, loading adapter weights from the Hub
print(f"Applying PEFT adapter '{ADAPTER_NAME_ON_HUB}' from Hub to the base model...")
model = PeftModel.from_pretrained(
    base_model,
    HUB_REPO_ID, # Main repository ID
    subfolder=ADAPTER_NAME_ON_HUB, # Specify the subfolder for the adapter
    token=HF_TOKEN
)
model.eval() # Set the model to evaluation mode
print("PEFT adapter applied. Model is ready.")

# Set pad_token if not already set
if tokenizer.pad_token is None:
    print("Tokenizer pad_token is None. Setting it to eos_token.")
    tokenizer.pad_token = tokenizer.eos_token

# 'model' and 'tokenizer' are now ready for use.
ADAPTER_PATH = f"{HUB_REPO_ID}/{ADAPTER_NAME_ON_HUB}"

print(f"\nModel and tokenizer for '{ADAPTER_PATH}' loaded successfully from Hugging Face Hub.")

Loading adapter 'nlistral-7b-qlora-ablation1-best' from Hub repo 'jd0g/nlistral-7b-qlora'...
Base model specified in adapter config: mistralai/Mistral-7B-v0.3
Loading base model 'mistralai/Mistral-7B-v0.3' with 4-bit quantization on GPU 0...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Base model loaded.
Loading tokenizer from Hub: 'jd0g/nlistral-7b-qlora' (subfolder: 'nlistral-7b-qlora-ablation1-best')...
Tokenizer loaded.
Resizing token embeddings of base model from 32768 to 32769.
Applying PEFT adapter 'nlistral-7b-qlora-ablation1-best' from Hub to the base model...
PEFT adapter applied. Model is ready.

Model and tokenizer for 'jd0g/nlistral-7b-qlora/nlistral-7b-qlora-ablation1-best' loaded successfully from Hugging Face Hub.


### (OPTIONAL) Download ALL Fine-Tuned QLoRA Adaptors Trained, Load Mistral Base Model & Tokenizer
If you want to download all the models locally and compare different ablations, you can run the below two cells in this section. Skip if you already ran the above cell

In [61]:
import os
from pathlib import Path
from huggingface_hub import hf_hub_download
from dotenv import load_dotenv; load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")
REPO_ID = "jd0g/nlistral-7b-qlora"
LOCAL_MODEL_DIR = Path("models")
model_path = "nlistral-7b-qlora-ablation1-best"
local_dir = "nlistral-ablation1"

model_dir = LOCAL_MODEL_DIR / local_dir
os.makedirs(model_dir, exist_ok=True)

for file_name in ["adapter_config.json", "adapter_model.safetensors", "special_tokens_map.json", "tokenizer.model", "tokenizer.json", "tokenizer_config.json"]:
    file_path = f"{model_path}/{file_name}"
    print(f"Downloading: {file_path}")
    temp_file = hf_hub_download(repo_id=REPO_ID, filename=file_path, token=HF_TOKEN, local_dir=LOCAL_MODEL_DIR)
    dest_path = model_dir / file_name
    if os.path.dirname(temp_file) != str(model_dir): 
        import shutil; shutil.copy2(temp_file, dest_path); os.remove(temp_file)

print(f"✅ Model downloaded to: {model_dir}")

Downloading: nlistral-7b-qlora-ablation1-best/adapter_config.json
Downloading: nlistral-7b-qlora-ablation1-best/adapter_model.safetensors
Downloading: nlistral-7b-qlora-ablation1-best/special_tokens_map.json
Downloading: nlistral-7b-qlora-ablation1-best/tokenizer.model
Downloading: nlistral-7b-qlora-ablation1-best/tokenizer.json
Downloading: nlistral-7b-qlora-ablation1-best/tokenizer_config.json
✅ Model downloaded to: models/nlistral-ablation1


In [33]:
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from dotenv import load_dotenv

# --- Configuration ---
ADAPTER_PATH = "models/nlistral-ablation1"  # Path to your adapter checkpoint directory
GPU_ID = 0                                 # GPU to load the model on
# ---

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

adapter_config_file = os.path.join(ADAPTER_PATH, "adapter_config.json")
with open(adapter_config_file, 'r') as f:
    adapter_config = json.load(f)
BASE_MODEL_ID = adapter_config.get("base_model_name_or_path", "mistralai/Mistral-7B-v0.3")

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=quantization_config,
    device_map={" quenched": GPU_ID},
    trust_remote_code=True,
    token=HF_TOKEN
)

tokenizer = AutoTokenizer.from_pretrained(
    ADAPTER_PATH,
    use_fast=True,
    padding_side="left",
    token=HF_TOKEN
)

if len(tokenizer) != base_model.config.vocab_size:
    base_model.resize_token_embeddings(len(tokenizer))

model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model.eval()

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 'model' and 'tokenizer' are now ready for use.

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


### Process a Single Input

In [62]:
from utils.prompts import FINETUNE_PROMPT
FINETUNE_PROMPT

premise = "The cat is on the rug."
hypothesis = "A feline is on the mat."

formatted_prompt = FINETUNE_PROMPT.format(
    premise=premise,
    hypothesis=hypothesis
)

input_text = f"[INST] {formatted_prompt} [/INST]"


inputs = tokenizer(input_text, return_tensors="pt").to(next(model.parameters()).device)

# Generate
with torch.no_grad():
    output_sequences = model.generate(**inputs, max_new_tokens=256, pad_token_id=tokenizer.eos_token_id)

# Decode and print only the generated part
generated_text = tokenizer.decode(output_sequences[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
generated_text

'{"thought_process": "step 1: the premise states that the cat is on the rug. a rug is a type of mat, often used for floor coverings. step 2: the hypothesis suggests that a feline is on the mat. since a mat can refer to a rug, and the cat is on the rug, it can be inferred that a feline is on the mat. step 3: based on the logical reasoning and lack of contradictory facts, the hypothesis can be inferred from the premise.", "predicted_label": 1} '

### Process outputs from an input CSV file

In [63]:
import pandas as pd
import torch
import os


CSV_FILE_PATH = 'data/original_data/sample.csv'
MAX_NEW_TOKENS_PER_ITEM = 256 

input_model_name = os.path.splitext(os.path.basename(ADAPTER_PATH))[0]
input_data_name = os.path.splitext(os.path.basename(CSV_FILE_PATH))[0]
OUTPUT_CSV_FILENAME = f"{input_model_name}_{input_data_name}_predictions.csv"
OUTPUT_COLUMN_NAME = 'output' # As per your script

# --- Script ---
df = pd.read_csv(CSV_FILE_PATH) # Processing the whole CSV as per updated request
df = pd.read_csv(CSV_FILE_PATH)[:5] # If you want to test with first 10 rows

model_device = next(model.parameters()).device
results_list = []

print(f"Processing {len(df)} rows from {CSV_FILE_PATH} to create {OUTPUT_CSV_FILENAME} using FINETUNE_PROMPT.")

for index, row in df.iterrows():
    premise = str(row['premise'])
    hypothesis = str(row['hypothesis'])

    # Format the prompt using FINETUNE_PROMPT
    formatted_core_prompt = FINETUNE_PROMPT.format(
        premise=premise,
        hypothesis=hypothesis
    )
    # Add Mistral instruction tags
    input_text = f"[INST] {formatted_core_prompt} [/INST]"

    max_model_length = getattr(model.config, 'max_position_embeddings', 512) 
    max_input_length = max_model_length - MAX_NEW_TOKENS_PER_ITEM
    
    inputs = tokenizer(
        input_text, 
        return_tensors="pt", 
        truncation=True, 
        max_length=max_input_length 
    ).to(model_device)

    with torch.no_grad():
        output_sequences = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS_PER_ITEM,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False # For deterministic output (recommended for structured tasks)
        )
    
    # Decode only the newly generated text
    generated_text = tokenizer.decode(output_sequences[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    results_list.append(generated_text)
    print(f"  Processed {index + 1}/{len(df)} items.")
    
    del inputs, output_sequences
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

df[OUTPUT_COLUMN_NAME] = results_list
df.to_csv(OUTPUT_CSV_FILENAME, index=False)

print(f"\nFinished processing. Output saved to: {OUTPUT_CSV_FILENAME}")
df.head()

Processing 5 rows from data/original_data/sample.csv to create nlistral-7b-qlora-ablation1-best_sample_predictions.csv using FINETUNE_PROMPT.
  Processed 1/5 items.
  Processed 2/5 items.
  Processed 3/5 items.
  Processed 4/5 items.
  Processed 5/5 items.

Finished processing. Output saved to: nlistral-7b-qlora-ablation1-best_sample_predictions.csv


,premise,hypothesis,label,output
0,a Bombay looks uh i guess the closest thing to...,Is it because Halloween cat caricatures and Bo...,1,"{""thought_process"": ""step 1: the premise descr..."
1,Sailing north to find and claim islands for th...,Columbus traveled all over the world for the s...,1,"{""thought_process"": ""step 1: the premise descr..."
2,his conviction was later overturned on appeal.,He was found guilty of Conspiracy to Rob but h...,1,"{""thought_process"": ""step 1: in the premise, i..."
3,"In practice, and depending on the language, th...","In practice, and depending on the language, th...",1,"{""thought_process"": ""step 1: the premise state..."
4,"You tread on her foot, or pick up her handkerc...","Don't tread on her foot, or pick up her handke...",1,"{""thought_process"": ""step 1: in the premise, t..."


In [44]:
import pandas as pd
import torch
import os
from tqdm.notebook import tqdm # For a progress bar

# --- Configuration ---
CSV_FILE_PATH = 'data/original_data/sample.csv'
MAX_NEW_TOKENS_PER_ITEM = 256
BATCH_SIZE = 8

# Assumes ADAPTER_PATH is defined in the notebook scope from a previous cell
input_model_name = os.path.splitext(os.path.basename(ADAPTER_PATH))[0]
input_data_name = os.path.splitext(os.path.basename(CSV_FILE_PATH))[0]
OUTPUT_CSV_FILENAME = f"{input_model_name}_{input_data_name}_predictions_batched.csv"
OUTPUT_COLUMN_NAME = 'output'

# --- Script ---
df = pd.read_csv(CSV_FILE_PATH)
model_device = next(model.parameters()).device # Assumes model is loaded
all_results = []

print(f"Processing {len(df)} rows from {CSV_FILE_PATH} in batches of {BATCH_SIZE} to create {OUTPUT_CSV_FILENAME}.")
print(f"Using FINETUNE_PROMPT (first 100 chars): {FINETUNE_PROMPT[:100]}...")
max_model_length = getattr(model.config, 'max_position_embeddings', 512)
max_input_length = max_model_length - MAX_NEW_TOKENS_PER_ITEM

# Process data in batches
for i in tqdm(range(0, len(df), BATCH_SIZE), desc="Processing Batches"):
    batch_df = df.iloc[i:i+BATCH_SIZE]
    batch_prompts = []

    for index, row in batch_df.iterrows():
        premise = str(row['premise'])
        hypothesis = str(row['hypothesis'])
        # Assumes FINETUNE_PROMPT is defined and loaded
        formatted_core_prompt = FINETUNE_PROMPT.format(premise=premise, hypothesis=hypothesis)
        input_text = f"[INST] {formatted_core_prompt} [/INST]"
        batch_prompts.append(input_text)
    
    # Batch tokenization (assumes tokenizer is loaded)
    inputs = tokenizer(
        batch_prompts, 
        return_tensors="pt", 
        padding=True,
        truncation=True, 
        max_length=max_input_length 
    ).to(model_device)

    # Batch generation
    with torch.no_grad():
        output_sequences = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS_PER_ITEM,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False
        )
    
    batch_generated_texts = []
    prompt_len_in_batch = inputs["input_ids"].shape[1]
    for j in range(output_sequences.shape[0]):
        generated_part = tokenizer.decode(output_sequences[j][prompt_len_in_batch:], skip_special_tokens=True)
        batch_generated_texts.append(generated_part)
        
    all_results.extend(batch_generated_texts)
    
    del inputs, output_sequences, batch_prompts
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

df[OUTPUT_COLUMN_NAME] = all_results
df.to_csv(OUTPUT_CSV_FILENAME, index=False)

print(f"\nFinished processing. Output saved to: {OUTPUT_CSV_FILENAME}")
df.head()

Processing 100 rows from data/original_data/sample.csv in batches of 8 to create nlistral-ablation1_sample_predictions_batched.csv.


Processing Batches:   0%|          | 0/13 [00:00<?, ?it/s]


Finished processing. Output saved to: nlistral-ablation1_sample_predictions_batched.csv


,premise,hypothesis,label,output
0,a Bombay looks uh i guess the closest thing to...,Is it because Halloween cat caricatures and Bo...,1,"{""thought_process"": ""step 1: the premise descr..."
1,Sailing north to find and claim islands for th...,Columbus traveled all over the world for the s...,1,"{""thought_process"": ""step 1: the premise descr..."
2,his conviction was later overturned on appeal.,He was found guilty of Conspiracy to Rob but h...,1,"{""thought_process"": ""step 1: in the premise, i..."
3,"In practice, and depending on the language, th...","In practice, and depending on the language, th...",1,"{""thought_process"": ""step 1: the premise state..."
4,"You tread on her foot, or pick up her handkerc...","Don't tread on her foot, or pick up her handke...",1,"{""thought_process"": ""step 1: in the premise, t..."


### Batch Processing with JSON Extraction for predicted_label

In [46]:
import pandas as pd
import torch
import os
import json
import re
from tqdm.notebook import tqdm

# --- ASSUMED FROM PREVIOUS CELLS: ADAPTER_PATH, FINETUNE_PROMPT, model, tokenizer ---

# --- Configuration ---
CSV_FILE_PATH = 'data/original_data/sample.csv'
MAX_NEW_TOKENS_PER_ITEM = 256
BATCH_SIZE = 8 
# ---

input_model_name = os.path.splitext(os.path.basename(ADAPTER_PATH))[0]
input_data_name = os.path.splitext(os.path.basename(CSV_FILE_PATH))[0]

FULL_OUTPUT_CSV_FILENAME = f"{input_model_name}_{input_data_name}_full_outputs_batched.csv"
FINAL_PREDICTIONS_CSV_FILENAME = f"{input_model_name}_{input_data_name}_final_predictions.csv"
OUTPUT_COLUMN_NAME = 'generated_full_output'
PREDICTION_COLUMN_NAME = 'prediction'

def extract_simple_prediction(output_text):
    output_text = str(output_text).strip()
    try:
        match = re.search(r'\{.*\}', output_text, re.DOTALL)
        if match:
            response_dict = json.loads(match.group(0))
            label_value = response_dict.get('predicted_label', response_dict.get('label'))
            if label_value is not None: return int(label_value)
    except (json.JSONDecodeError, TypeError, ValueError): pass

    for pattern in [r'predicted_label\s*:\s*([01])', 
                    r'final\s+label\s*:\s*([01])', 
                    r'final\s+prediction\s*:\s*([01])']:
        match = re.search(pattern, output_text.lower())
        if match: return int(match.group(1))
    return None

df = pd.read_csv(CSV_FILE_PATH)
model_device = next(model.parameters()).device
all_full_outputs = []

max_model_length = getattr(model.config, 'max_position_embeddings', 512)
max_input_length = max_model_length - MAX_NEW_TOKENS_PER_ITEM

for i in tqdm(range(0, len(df), BATCH_SIZE), desc="Batch Processing"):
    batch_df = df.iloc[i:i+BATCH_SIZE]
    batch_prompts = [
        f"[INST] {FINETUNE_PROMPT.format(premise=str(row['premise']), hypothesis=str(row['hypothesis']))} [/INST]"
        for _, row in batch_df.iterrows()
    ]
    
    inputs = tokenizer(
        batch_prompts, return_tensors="pt", padding=True,
        truncation=True, max_length=max_input_length 
    ).to(model_device)

    with torch.no_grad():
        output_sequences = model.generate(
            **inputs, max_new_tokens=MAX_NEW_TOKENS_PER_ITEM,
            pad_token_id=tokenizer.eos_token_id, do_sample=False
        )
    
    prompt_len_in_batch = inputs["input_ids"].shape[1]
    all_full_outputs.extend([
        tokenizer.decode(output_sequences[j][prompt_len_in_batch:], skip_special_tokens=True)
        for j in range(output_sequences.shape[0])
    ])
    
    del inputs, output_sequences, batch_prompts
    if torch.cuda.is_available(): torch.cuda.empty_cache()

df[OUTPUT_COLUMN_NAME] = all_full_outputs
df.to_csv(FULL_OUTPUT_CSV_FILENAME, index=False)
print(f"\nFull outputs saved to: {FULL_OUTPUT_CSV_FILENAME}")

extracted_predictions = df[OUTPUT_COLUMN_NAME].apply(extract_simple_prediction)
predictions_df = pd.DataFrame({PREDICTION_COLUMN_NAME: extracted_predictions})
predictions_df.to_csv(FINAL_PREDICTIONS_CSV_FILENAME, index=False)
print(f"Final predictions saved to: {FINAL_PREDICTIONS_CSV_FILENAME}")

print("\nDone. Displaying head of final predictions:")
print(predictions_df.head())

Processing 100 rows from data/original_data/sample.csv, BATCH_SIZE=16.
Output (full): nlistral-ablation1_sample_full_outputs_batched.csv
Output (preds): nlistral-ablation1_sample_final_predictions.csv


Batch Processing:   0%|          | 0/7 [00:00<?, ?it/s]


Full outputs saved to: nlistral-ablation1_sample_full_outputs_batched.csv
Final predictions saved to: nlistral-ablation1_sample_final_predictions.csv

Done. Displaying head of final predictions:
   prediction
0           1
1           1
2           1
3           1
4           1


In [50]:
pd.read_csv(FINAL_PREDICTIONS_CSV_FILENAME)

,prediction
0,1
1,1
2,1
3,1
4,1
...,...
95,0
96,0
97,0
98,1


In [51]:
pd.read_csv(FULL_OUTPUT_CSV_FILENAME)

,premise,hypothesis,label,generated_full_output
0,a Bombay looks uh i guess the closest thing to...,Is it because Halloween cat caricatures and Bo...,1,"{""thought_process"": ""step 1: the premise descr..."
1,Sailing north to find and claim islands for th...,Columbus traveled all over the world for the s...,1,"{""thought_process"": ""step 1: the premise descr..."
2,his conviction was later overturned on appeal.,He was found guilty of Conspiracy to Rob but h...,1,"{""thought_process"": ""step 1: in the premise, i..."
3,"In practice, and depending on the language, th...","In practice, and depending on the language, th...",1,"{""thought_process"": ""step 1: the premise state..."
4,"You tread on her foot, or pick up her handkerc...","Don't tread on her foot, or pick up her handke...",1,"{""thought_process"": ""step 1: in the premise, t..."
...,...,...,...,...
95,It is referred to as the Warwickshire Avon Avo...,It is also called the Warwickshire Avon or Sha...,1,"{""thought_process"": ""step 1: the premise state..."
96,i still haven't checked out to see if they do ...,They don't make the 18.,0,"{""thought_process"": ""step 1: in the premise, t..."
97,"From Adelaide, South Australia, Hele played cl...","From Adelaide, South Australia, Hele played cl...",0,"{""thought_process"": ""step 1: the premise state..."
98,"Poirot unlocked the door and, upon seeing her ...",Poirot recognized her distress the moment he g...,1,"{""thought_process"": ""step 1: the premise state..."
